<img src="https://climatecompatiblegrowth.com/wp-content/uploads/OnSSET_text.png" height="150" align="center">

<img src="https://avatars.githubusercontent.com/u/62061836?s=200&v=4" height="80" align="center">

<img src="https://onstove-documentation.readthedocs.io/en/latest/_images/kth_logo.svg" height="80" align="center">

<img src="https://www.ukaiddirect.org/images/logo.png" height="70" align="center">

# 👋 Welcome to the Starter Data Kit for `OnSSET`!
This notebook will help you download the starter datasets needed to run a basic `OnSSET` analysis. It will allow you to download the geospatial data required as well as the socio- and techno-economic data of your country of interest. There are different data sources available and more will be added in future versions of the tool.

Please follow the steps below, settup the analysis and choose the country and datasets to download.

In [ ]:
#%%capture
# @title # 📦 Import required packages {"vertical-output":true,"display-mode":"form"}
# @markdown Run this cell to install all required packages and setup the session by clicking on the run button (▶️) on the upper left corner of the cell.
# @markdown > ⚠️ This may take several minutes.

import subprocess
from IPython.display import clear_output

def run_quiet_shell(command):
    # Start the process
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    # Read output line by line as it happens
    for line in process.stdout:
        clear_output(wait=True)
        print(line.strip())

    process.wait()

# Usage:
run_quiet_shell("pip install --ignore-installed blinker tqdm --upgrade")
run_quiet_shell("pip install git+https://github.com/ClimateCompatibleGrowth/Starter-Data-Kits.git --upgrade")
run_quiet_shell("pip install ipyleaflet leafmap localtileserver --upgrade")

from google.colab import userdata
import os
# import awswrangler as wr
import ipywidgets as widgets
from ipywidgets import Dropdown, Layout, SelectMultiple, Button, HBox, VBox
from IPython.display import display
from google.colab.output import eval_js
from IPython.display import HTML
import starterkits as sdk
import geopandas as gpd
from tqdm.notebook import tqdm
import glob
from google.colab import output
import time

print('Setup complete')

In [ ]:
# @title # 🛠️ Setup and select datasets {"run":"auto","vertical-output":true}
# @markdown Run this cell to display the posible datasets to download for the OnSSET analysis.
# @markdown First fill the login information to the required databases and select
# @markdown the country of analysis, then click on the run botton (▶️) on the upper left corner.
# @markdown ## 🌐 Database connection
# @markdown ### 🛰️ NASA Earth database
# @markdown The Nasa Earth database is **required** to download landcover type MCD12Q1 data from the MODIS sensor,
# @markdown and elevation data (DEM) at different resolutions.
# @markdown You will need a username and a password from NASA Earth Data, please go to
# @markdown https://urs.earthdata.nasa.gov/home create a free account and fill the login credentials below.
NASA_username = "" # @param {"type":"string","placeholder":"👤 Enter NASA Earth username"}
NASA_password = "" # @param {"type":"string","placeholder":"🔒 Enter NASA Earth password"}

## @markdown ### OpenTopography database
## @markdown Digital Elevation Model data (DEM) can be downloaded from the OpenTopography
## @markdown project, for which you will need an API key. Please go to https://opentopography.org
## @markdown create a free account and generate an API key. Copy the key and paste it in the field below.
#Topo_API_key = "" # @param {"type":"string","placeholder":"Enter OpenTopography API key"}

## @markdown > ⚠️ **Note:** If you are using the NASA Earth database for Elevation data,
## @markdown you do not need to use OpenTopography.

# @markdown ## 🌍 Country and datasets selection
# @markdown In the dropdown below select the country of interest and several options will be
# @markdown displayed of posible datasets to download. Select which datasets and sources
# @markdown you want to use and click on the `Get data` button.

Country = "Select country" # @param ["Select country", 'AGO', 'BDI', 'BEN', 'BFA', 'BWA', 'CAF', 'CIV', 'CMR', 'COD', 'COG', 'DJI', 'ERI', 'ETH', 'GAB', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'KEN', 'LBR', 'LSO', 'MDG', 'MLI', 'MOZ', 'MRT', 'MWI', 'NAM', 'NER', 'NGA', 'RWA', 'SDN', 'SEN', 'SLE', 'SOM', 'SSD', 'SWZ', 'TCD', 'TGO', 'TZA', 'UGA', 'ZAF', 'ZMB', 'ZWE']

fa_loader = HTML('''<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"> ''')

# FIX SCROLLBAR: This tells Colab to expand the output area
output.eval_js('google.colab.output.setIframeHeight(0, true, {maxHeight: 3000})')

font_style = widgets.HTML("""
<style>
    /* --- 1. GLOBAL & TEXT STYLES --- */
    .widget-label, .widget-checkbox > label > span {
        color: var(--colab-primary-text-color) !important;
        font-family: "Segoe UI", Tahoma, sans-serif !important;
    }
    .widget-checkbox > label > span {
        font-size: 16px !important;
        font-weight: 600 !important;
        padding-left: 5px;
    }
    .widget-label {
        font-size: 15px !important;

    }
    .widget-html { font-size: 14px !important; }
    .jupyter-button { font-weight: bold !important; letter-spacing: 0.5px; }
    .jupyter-button i { margin-right: 8px; }

    /* This kills the accordion lines once and for all */
    .borderless-accordion, .borderless-accordion * {
        border: none !important;
        box-shadow: none !important;
        outline: none !important;
        padding-left: 0px !important;
        padding-right: 0px !important;
    }

    .widget-vbox {
        max-width: 1200px !important;
        overflow-x: hidden !important;
    }

    /* --- 3. ACCORDION HEADER STYLING --- */
    /* We must redefine this because the Nuclear Fix above stripped it */
    .p-Collapse-header, .lm-Collapse-header {
        color: #1a73e8 !important;
        padding-left: 0px !important;
        font-family: 'Segoe UI', Roboto, sans-serif;
        border-bottom: 2px solid #d1d5db !important;

        font-size: 20px !important;
        font-weight: 800 !important;

        /* Shadow for depth */
        transition: all 0.2s ease !important;
    }
    .p-Collapse-header:hover, .lm-Collapse-header:hover {
        background-color: #e8f0fe !important;
    }

    /* --- RESTORE BORDERS FOR DROPDOWNS & CONTROLS --- */
    /* Targets the inner select box of dropdowns and the radio containers */
    .borderless-accordion select,
    .borderless-accordion .widget-dropdown > select {
        border: 1px solid var(--colab-border-color) !important;
        border-radius: 4px !important;
        background-color: var(--colab-primary-surface-color) !important;
        padding: 4px !important;
    }

    /* Subtle shadow for the dropdown to make it look interactive */
    .widget-dropdown > select {
        box-shadow: inset 0 1px 2px rgba(0,0,0,0.05) !important;
    }

    .dataset-card .widget-html span {
        padding-left: 8px !important;
        padding-right: 8px !important;
        display: inline-block; /* Ensures padding is respected */
    }

    /* --- 4. DATASET CARD PROTECTION --- */
    /* This restores the "Card" look for the items inside the accordion */
    .dataset-card {
        background-color: var(--colab-primary-surface-color) !important;
        border: 1px solid var(--colab-border-color) !important;
        border-radius: 12px !important;
        padding: 18px !important;
        margin: 10px 0px !important;
        box-shadow: 0 2px 4px rgba(0,0,0,0.08) !important;
    }

    /* --- 5. SPECIAL CONTAINERS --- */
    .select-all-container {
        background-color: var(--colab-secondary-surface-color) !important;
        border: 2px dashed var(--colab-border-color) !important;
        border-radius: 10px !important;
        padding: 10px 15px !important;
        margin-bottom: 20px !important;
    }
    .select-all-container .widget-label span {
        font-size: 17px !important;
        font-weight: 800 !important;
        color: #1a73e8 !important;
    }

    .widget-output {
        box-sizing: border-box !important;
        padding-right: 2px !important; /* Tiny gap to let the border breathe */
    }

    /* --- 6. ANIMATIONS --- */
    .fa-spinner { animation: spin 2s linear infinite; }
    @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } }
</style>
""")

def create_dataset_card(label, description_text, dropdown_options=None,
                        dropdown_value=None, radio_label=None,
                        radio_options=None, tag="Optional"):
    """
    Creates a fully styled dataset card.
    Returns: (card_widget, checkbox_widget)
    """
    # 1. Create the checkbox
    checkbox = widgets.Checkbox(description=label, value=False, indent=False)

    # 2. Status Badge (Tag)
    tag_color = "#1a73e8" if tag == "Required" else "#5f6368"
    tag_html = f"<span style='float:right; font-size:10px; color:white; background:{tag_color}; padding:2px 8px; border-radius:10px; font-weight:bold; text-transform:uppercase; margin-left: 10px;'>{tag}</span>"

    # 3. Description with Tag
    desc_content = f"{tag_html}<div style='line-height:1.4; margin-top:5px; padding-right: 70px;'>ℹ️ {description_text}</div>"
    desc_widget = widgets.HTML(value=desc_content, layout=Layout(margin='5px 0 10px 0'))

    # 4. Assemble Card
    card_items = [checkbox, desc_widget]
    if dropdown_options:
      controls = []
      if dropdown_value is None:
        dropdown_value = dropdown_options[0]
      dropdown = widgets.Dropdown(
          description='Source',
          options=dropdown_options,
          disabled=True,
          value=dropdown_value
      )
      controls.append(dropdown)
      if radio_options:
        radio = widgets.RadioButtons(
            options=radio_options,
            description=radio_label,
            disabled=True
        )
        controls.append(radio)
      else:
        radio = None
      ctrl_box = HBox(controls, layout=Layout(grid_gap='20px', padding='0 0 0 20px'))
      card_items.append(ctrl_box)
    else:
      dropdown = None
      radio = None

    card = VBox(card_items, layout=Layout(width='100%', padding='15px', overflow='visible'))
    card.add_class('dataset-card')

    return card, checkbox, dropdown, radio

# Create the widgets
log_output = widgets.Output(
    layout=Layout(
        width='99%',
        height='150px',      # Fixed height keeps the dashboard stable
        border='1px solid #e0e0e0',
        overflow_y='scroll', # Adds a scrollbar if logs get long
        margin='10px 0'
    )
)

title = widgets.HTML('<h1>📂 Choose the datasets to download in the options below</h1>')

all_checkbox = widgets.Checkbox(
    description='Select all', # Label
    value=False,                            # Default value
    disabled=False,                         # Whether to disable user changes
    indent=False                             # Align with other controls with a description
)

# --- Socio-Demographics ---
specs_card, specs_checkbox, _, _ = create_dataset_card(
    '💰 Socio-economic specifications',
    'Socio-economic specifications define the financial and ' +
    'behavioral parameters of the target population. This includes ' +
    'estimated household energy consumption tiers, affordability ' +
    '(willingness to pay), and projected GDP growth. These values ' +
    'are used to calculate the long-term economic viability and ' +
    'expected revenue for grid investments.',
    tag="Required"
)

pop_card, pop_checkbox, pop_dropdown, pop_radio = create_dataset_card(
    '👥 Population count',
    'Population count data provides the spatial distribution of ' +
    'inhabitants. This is critical for estimating electricity demand ' +
    'and identifying clusters for potential microgrid or grid ' +
    'extension projects.',
    dropdown_options=['Worldpop'],
    radio_options=['100m', '1km'],
    radio_label='Resolution:',
    tag="Required"
)

# --- Environmental ---
dem_card, dem_checkbox, dem_dropdown, _ = create_dataset_card(
    '🏔️ Elevation model',
    'Digital Elevation Model data (DEM), is used ' +
    'to determine terrain slope, and both terrain ' +
    'slope and elevation are used to specify grid extension suitability. ' +
    'There are several sources available, but a `SRTMGL3` with ' +
    'a resolution of ~90m is often enough, preventing ' +
    'large computation times.',
    dropdown_options=sdk.apis.elevation_datasets['NASA'].keys(),
    dropdown_value='SRTMGL3'
)

solar_card, solar_checkbox, solar_dropdown, _ = create_dataset_card(
    '☀️ Solar irradiation',
    'Solar irradiation (GHI) data measures the energy received ' +
    'from the sun. This is the primary input for determining ' +
    'the feasibility and sizing of solar photovoltaic (PV) ' +
    'installations.',
    dropdown_options=['Global Wind Atlas'],
    tag="Required"
)

wind_card, wind_checkbox, wind_dropdown, wind_radio = create_dataset_card(
    '🌬️ Wind speed',
    'Wind speed data at various heights (hub heights) is used to ' +
    'assess the technical potential for wind energy generation. ' +
    'Higher resolution data helps pinpoint areas where turbines ' +
    'will be most productive.',
    dropdown_options=['Global Wind Atlas'],
    radio_options=['100', '150', '200'],
    radio_label='Height (m):',
    tag="Required"
)

land_card, land_checkbox, land_dropdown, _ = create_dataset_card(
    '🌳 Land cover',
    'Landcover data classifies land into 17 ' +
    'specific categories. Different categories ' +
    'imply different penalties on the grid extension.',
    dropdown_options = ['MODIS (MCD12Q1)']
)

# --- Infrastructure ---
mv_card, mv_checkbox, mv_dropdown, _ = create_dataset_card(
    '⚡ Medium voltage lines',
    'Medium voltage (MV) infrastructure data represents the existing ' +
    'power grid. Knowing the location of current lines allows the model ' +
    'to calculate the shortest and most cost-effective connection ' +
    'points for unelectrified areas.',
    dropdown_options=['GridFinder'],
    tag="Required"
)

ntl_card, ntl_checkbox, ntl_dropdown, _ = create_dataset_card(
    '🌃 Nighttime lights',
    'Nighttime light (NTL) intensity is used as a proxy for ' +
    'existing electrification and economic activity. It helps ' +
    'validate population data and identifies areas that are ' +
    'likely already connected to the grid.',
    dropdown_options=['WorldPop covariates derived from VNL 2.1/2.2'],
    tag="Required"
)

roads_card, roads_checkbox, roads_dropdown, _ = create_dataset_card(
    '🛣️ Roads',
    'Road network data is a key proxy for accessibility. ' +
    'Proximity to roads significantly reduces the logistical ' +
    'costs of transporting equipment and maintaining power ' +
    'infrastructure.',
    dropdown_options=['OSM road network'],
)

travel_card, travel_checkbox, travel_dropdown, _ = create_dataset_card(
    '🚚 Traveltime',
    'Travel time data estimates the duration required to reach any given ' +
    'location from the nearest urban center or port. It accounts for road ' +
    'quality, terrain slope, and land cover, providing a metric for ' +
    'estimating transport costs of diesel for off-grid generators.',
    dropdown_options=['Malarian Atlas'],
)

download = Button(description='Get data',
                  icon= 'cloud-download',
                  layout=Layout(width='200px', height='40px'),
                  button_style='primary')

def on_all_checkbox_changed(i):
  if all_checkbox.value:
    specs_checkbox.value = True
    pop_checkbox.value = True
    mv_checkbox.value = True
    wind_checkbox.value = True
    solar_checkbox.value = True
    dem_checkbox.value = True
    land_checkbox.value = True
    ntl_checkbox.value = True
    roads_checkbox.value = True
    travel_checkbox.value = True
  else:
    specs_checkbox.value = False
    pop_checkbox.value = False
    mv_checkbox.value = False
    wind_checkbox.value = False
    solar_checkbox.value = False
    dem_checkbox.value = False
    land_checkbox.value = False
    ntl_checkbox.value = False
    roads_checkbox.value = False
    travel_checkbox.value = False

def on_pop_checkbox_changed(i):
  if pop_checkbox.value:
    pop_dropdown.disabled = False
    pop_radio.disabled = False
  else:
    pop_dropdown.disabled = True
    pop_radio.disabled = True

def on_mv_checkbox_changed(i):
  if mv_checkbox.value:
    mv_dropdown.disabled = False
  else:
    mv_dropdown.disabled = True

def on_wind_checkbox_changed(i):
  if wind_checkbox.value:
    wind_dropdown.disabled = False
    wind_radio.disabled = False
  else:
    wind_dropdown.disabled = True
    wind_radio.disabled = True

def on_solar_checkbox_changed(i):
  if solar_checkbox.value:
    solar_dropdown.disabled = False
  else:
    solar_dropdown.disabled = True

def on_dem_checkbox_changed(i):
  if dem_checkbox.value:
    dem_dropdown.disabled = False
  else:
    dem_dropdown.disabled = True

def on_land_checkbox_changed(i):
  if land_checkbox.value:
    land_dropdown.disabled = False
  else:
    land_dropdown.disabled = True

def on_ntl_checkbox_changed(i):
  if ntl_checkbox.value:
    ntl_dropdown.disabled = False
  else:
    ntl_dropdown.disabled = True

def on_roads_checkbox_changed(i):
  if roads_checkbox.value:
    roads_dropdown.disabled = False
  else:
    roads_dropdown.disabled = True

def on_travel_checkbox_changed(i):
  if travel_checkbox.value:
    travel_dropdown.disabled = False
  else:
    travel_dropdown.disabled = True

def on_download_clicked(i):
  if Country == 'Select country':
    # Even warning messages can go to the log
    with log_output:
        print('⚠️ Please select a country first.')
    return

  # Clear previous logs for a fresh start
  log_output.clear_output()

  download.disabled = True              # Prevent double-clicks
  download.description = ' Fetching...' # Change text
  download.icon = 'spinner'             # Set to spinner icon
  download.button_style = 'warning'     # Change color to yellow/orange

  # Trap the tqdm and print statements inside the output widget
  with log_output:
    total_downloads = 1 # for boundaries

    if specs_checkbox.value:
      total_downloads += 1
    if pop_checkbox.value:
      total_downloads += 1
    if mv_checkbox.value:
      total_downloads += 1
    if wind_checkbox.value:
      total_downloads += 1
    if solar_checkbox.value:
      total_downloads += 1
    if dem_checkbox.value:
      total_downloads += 1
    if land_checkbox.value:
      total_downloads += 1
    if ntl_checkbox.value:
      total_downloads += 1
    if roads_checkbox.value:
      total_downloads += 1
    if travel_checkbox.value:
      total_downloads += 1

    with tqdm(total=total_downloads, desc="🔄 Downloading datasets") as pbar:
      pbar.set_description("🔄 Downloading boundaries")
      sdk.apis.get_boundaries(Country)
      pbar.update(1)

      if specs_checkbox.value:
        pbar.set_description("🔄 Downloading socio-economic specifications")
        sdk.apis.get_specs(Country)
        pbar.update(1)
      if pop_checkbox.value:
        pbar.set_description("🔄 Downloading population data")
        sdk.apis.get_population_data(Country, pop_radio.value)
        pbar.update(1)
      if mv_checkbox.value:
        pbar.set_description("🔄 Downloading medium voltage lines")
        sdk.apis.get_power_lines(Country)
        pbar.update(1)
      if wind_checkbox.value:
        pbar.set_description("🔄 Downloading wind speed data")
        sdk.apis.get_wind_data(Country, wind_radio.value)
        pbar.update(1)
      if solar_checkbox.value:
        pbar.set_description("🔄 Downloading solar irradiation data")
        sdk.apis.get_solar_data(Country)
        pbar.update(1)
      if dem_checkbox.value:
        pbar.set_description("🔄 Downloading elevation model data")
        sdk.apis.get_dem_data(country=Country, database='Nasa Earth',
                              nasa_username=NASA_username, nasa_password=NASA_password,
                              dem_type=dem_dropdown.value)
        pbar.update(1)
      if land_checkbox.value:
        pbar.set_description("🔄 Downloading land cover data")
        sdk.apis.get_landcover_data(country=Country,
                                    username=NASA_username, password=NASA_password)
        pbar.update(1)
      if ntl_checkbox.value:
        pbar.set_description("🔄 Downloading nighttime lights data")
        sdk.apis.get_ntl_data(Country)
        pbar.update(1)
      if roads_checkbox.value:
        pbar.set_description("🔄 Downloading roads data")
        sdk.apis.get_roads(Country) # , roads_types.value)
        pbar.update(1)
      if travel_checkbox.value:
        pbar.set_description("🔄 Downloading traveltime data")
        sdk.apis.get_traveltime_data(Country) # , roads_types.value)
        pbar.update(1)
      print('\n\033[1m✅ Data downloaded succesfully to session 🎉\033[0m')

  download.icon = 'check'               # Change to checkmark
  download.description = ' Data Ready'
  download.button_style = 'success'     # Change color to green

  time.sleep(3)

  download.description = 'Get Data'
  download.icon = 'cloud-download'
  download.button_style = 'primary'
  download.disabled = False             # Re-enable if you want them to download again

hr = widgets.HTML(value="<hr>")
space = widgets.HTML(value="<br>")

selection_summary = widgets.Label(value="0 datasets selected")

def update_summary(change):
    count = sum([specs_checkbox.value, pop_checkbox.value, mv_checkbox.value,
                 wind_checkbox.value, solar_checkbox.value, dem_checkbox.value,
                 land_checkbox.value, ntl_checkbox.value, roads_checkbox.value,
                 travel_checkbox.value])
    selection_summary.value = f"📊 Total datasets to fetch: {count}"

# Link all checkboxes to this function
for cb in [specs_checkbox, pop_checkbox, mv_checkbox, wind_checkbox,
           solar_checkbox, dem_checkbox, land_checkbox, ntl_checkbox, roads_checkbox,
           travel_checkbox]:
    cb.observe(update_summary, names='value')

# 3. Build UI Structure
master_toggle_card = VBox([all_checkbox], layout=Layout(width='100%', padding='10px')).add_class('select-all-container')

socio_acc = widgets.Accordion(children=[VBox([specs_card, pop_card])])
socio_acc.set_title(0, '👤 Socio-Demographics')
socio_acc.add_class('borderless-accordion')

env_acc = widgets.Accordion(children=[VBox([dem_card, solar_card, wind_card, land_card])])
env_acc.set_title(0, '🌍 Physical & Environment')
env_acc.add_class('borderless-accordion')

infra_acc = widgets.Accordion(children=[VBox([mv_card, ntl_card, roads_card, travel_card])])
infra_acc.set_title(0, '🏗️ Infrastructure')
infra_acc.add_class('borderless-accordion')

# 4. Observers (Reuse your existing on_change functions here)
# Example:
all_checkbox.observe(on_all_checkbox_changed, names='value')
pop_checkbox.observe(on_pop_checkbox_changed, names='value')
mv_checkbox.observe(on_mv_checkbox_changed, names='value')
wind_checkbox.observe(on_wind_checkbox_changed, names='value')
solar_checkbox.observe(on_solar_checkbox_changed, names='value')
dem_checkbox.observe(on_dem_checkbox_changed, names='value')
land_checkbox.observe(on_land_checkbox_changed, names='value')
ntl_checkbox.observe(on_ntl_checkbox_changed, names='value')
roads_checkbox.observe(on_roads_checkbox_changed, names='value')
travel_checkbox.observe(on_travel_checkbox_changed, names='value')
download.on_click(on_download_clicked)

# 5. Final Display
app_interface = VBox([
    title,
    space,
    master_toggle_card,
    socio_acc,
    env_acc,
    infra_acc,
    selection_summary,
    widgets.HTML("<b>🛰️ Console output</b>"),
    log_output,
    HBox([download], layout=Layout(justify_content='center', padding='20px'))
], layout=Layout(max_width='1050px', margin='0 auto'))

display(fa_loader, font_style, app_interface)

In [ ]:
# @title # 🗑️ Delete country data
# @markdown Run this cell to select a country to delete from the session.
# @markdown > ⚠️ If you downloaded data for a new country with the form above, you need to
# @markdown rerun this cell for it to appear in the dropdown.
# # delete data for a country
display(HTML('''<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"> '''))

def on_delete(b):
  try:
    !rm -rf /content/Data/{country_dropdown.value}
  except:
    pass
  if country_dropdown.value is not None:
    print(f'{country_dropdown.value} data deleted!')
    country_dropdown.options = get_countries()

def get_countries():
  countries = set()
  try:
    countries.update([files.split('/')[1] for files in glob.glob(f'Data/*')])
  except:
    pass
  return countries


country_dropdown = widgets.Dropdown(
                description='Country',
                options=get_countries(),
                disabled=False
)
delete = Button(description='Delete data',
                icon='trash',
                layout=Layout(width='200px', height='35px'),
                button_style='danger')

delete.on_click(on_delete)

display(HBox([country_dropdown, delete]))

In [ ]:
# @title # 📥 Download data
# @markdown Run this cell to download all files in the session to your computer.
import os
import zipfile
from google.colab import files as cfiles
import glob

def zip_and_download(folder_names, zip_filename="data.zip"):
  """Zips specified folders and downloads the zip file."""

  with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for folder_name in folder_names:
      for root, _, files in os.walk(folder_name):
        for file in files:
          file_path = os.path.join(root, file)
          arcname = os.path.relpath(file_path, '/content/') #Maintain folder structure
          zipf.write(file_path, arcname=arcname)

  cfiles.download(zip_filename)


# Specify the folders to be zipped
folders_to_zip = glob.glob('Data/*')
zip_and_download(folders_to_zip)